# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Built-in LangChain tools

In [ ]:
#!pip install -U wikipedia

In [3]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

In [4]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [5]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [6]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [9]:
agent("Can you build this triangle with sides a=2, b=3, c=10?")



> Entering new AgentExecutor chain...
Question: Can you build this triangle with sides a=2, b=3, c=10?
Thought: To determine if a triangle can be built with sides of lengths 2, 3, and 10, we need to check if the sum of the lengths of any two sides is greater than the length of the third side.
Action:
```
{
  "action": "Calculator",
  "action_input": "2+3"
}
```

Observation: Answer: 5
Thought:```
{
  "action": "Calculator",
  "action_input": "5>10"
}
```
Observation: Answer: False
Thought:```
{
  "action": "Calculator",
  "action_input": "3+10"
}
```
Observation: Answer: 13
Thought:```
{
  "action": "Calculator",
  "action_input": "2+10"
}
```
Observation: Answer: 12
Thought:```
{
  "action": "Calculator",
  "action_input": "13>2"
}
```
Observation: Answer: True
Thought:Final Answer: This triangle cannot be built with sides of lengths 2, 3, and 10.

> Finished chain.


{'input': 'Can you build this triangle with sides a=2, b=3, c=10?',
 'output': 'This triangle cannot be built with sides of lengths 2, 3, and 10.'}

In [11]:
agent("What is the derivation of funtion y=2x + 3x^2?")



> Entering new AgentExecutor chain...
Thought: To find the derivative of the function \( y = 2x + 3x^2 \), we can apply the rules of differentiation.
Action:
```
{
  "action": "Calculator",
  "action_input": "derivative of 2x + 3x^2"
}
```

ValueError: LLMMathChain._evaluate("
numexpr.evaluate("2 + 6*x")
") raised error: Expression numexpr.evaluate("2 + 6*x") has forbidden control characters.. Please try again with a valid numerical expression

## Wikipedia example

In [12]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Thought: I should use Wikipedia to find out which book Tom M. Mitchell wrote.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```
Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as t

Thought:Final Answer: The book written by Tom M. Mitchell is titled "Machine Learning."

> Finished chain.


## Python Agent

In [13]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [14]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [15]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
We can use the sorted() function in Python to sort the list of customers based on last name first and then first name.
Action: Python REPL
Action Input: customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
Observation: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

Thought:The customers have been sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

#### View detailed outputs of the chains

In [16]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

In [20]:
langchain.debug = False
agent.run(f"""Create a pandas df with timestamps as index \
and a sesonali variable x that has daily periodicity and print the full output.""") 



> Entering new AgentExecutor chain...
I need to import pandas and numpy to create the dataframe with timestamps as the index and a seasonal variable.
Action: Python REPL
Action Input:
```python
import pandas as pd
import numpy as np

# Create a datetime index
date_rng = pd.date_range(start='2022-01-01', end='2022-01-10', freq='D')

# Create a seasonal variable with daily periodicity
seasonal_data = np.sin(np.arange(len(date_rng)) * (2 * np.pi / 7))

# Create a DataFrame with timestamps as index and the seasonal variable
df = pd.DataFrame(data=seasonal_data, index=date_rng, columns=['seasonal_variable'])

print(df)
```
Observation:             seasonal_variable
2022-01-01       0.000000e+00
2022-01-02       7.818315e-01
2022-01-03       9.749279e-01
2022-01-04       4.338837e-01
2022-01-05      -4.338837e-01
2022-01-06      -9.749279e-01
2022-01-07      -7.818315e-01
2022-01-08      -2.449294e-16
2022-01-09       7.818315e-01
2022-01-10       9.749279e-01

Thought:The code successfull

'The pandas DataFrame with timestamps as the index and a seasonal variable with daily periodicity has been printed.'

## Define your own tool

In [ ]:
#!pip install DateTime

In [22]:
from langchain.agents import tool
from datetime import date

Docstring is used by the agent to understand when to use the function.

In [23]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [24]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [29]:
try:
    result = agent("whats the date today? What day number of the year is it?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Question: What's the date today? What day number of the year is it?
Thought: I can use the `time` tool to get today's date and then calculate the day number of the year.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2024-06-23
Thought:```
{
  "action": "Calculator",
  "action_input": "6*30+23"
}
```
Observation: Answer: 203
Thought:```
{
  "action": "Calculator",
  "action_input": "6*30+23"
}
```
Observation: Answer: 203
Thought:Final Answer: The date today is June 23, 2024. It is the 203rd day of the year.

> Finished chain.


Reminder: Download your notebook to you local computer to save your work.